In [1]:
import pandas as pd

In [7]:
from sqlalchemy import create_engine

In [9]:
engine.connect()

In [30]:
df_iter = pd.read_csv('green_tripdata.csv', iterator=True, chunksize=5000)

In [31]:
df=next(df_iter)

In [32]:
len(df)

5000

In [33]:
df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

In [10]:
print(pd.io.sql.get_schema(df, name="green_tripdata", con=engine))


CREATE TABLE green_tripdata (
	"VendorID" BIGINT, 
	lpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	lpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	store_and_fwd_flag TEXT, 
	"RatecodeID" BIGINT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	ehail_fee FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	payment_type BIGINT, 
	trip_type BIGINT, 
	congestion_surcharge FLOAT(53)
)




In [36]:
df.head(0).to_sql(name="green_tripdata", con=engine, if_exists="replace")

0

In [35]:
query = """
SELECT 1;
"""

pd.read_sql(query, con=engine)

,?column?
0,1


In [37]:
from time import time

In [38]:
total_rows_processed = 0
while True:
    try:
        t_start = time()

        df = next(df_iter)
        print(f"Processing chunk with {len(df)} rows")

        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

        df.to_sql(name='green_tripdata', con=engine, if_exists='append')

        total_rows_processed += len(df)
        t_end = time()
        print(f'Inserted another chunk, took {t_end - t_start:.3f} seconds')

    except StopIteration:
   
        print("All chunks have been processed. Total rows inserted: {total_rows_processed}")
        break

Processing chunk with 5000 rows
Inserted another chunk, took 22.894 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 19.212 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 35.699 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 25.828 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 24.941 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 55.604 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 25.627 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 18.617 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 11.355 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 10.183 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 29.859 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 15.235 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 17.468 seconds
Processing c

C:\Users\agyei\AppData\Local\Temp\ipykernel_11864\3629730013.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)


Inserted another chunk, took 10.086 seconds
Processing chunk with 5000 rows
Inserted another chunk, took 9.140 seconds
Processing chunk with 1386 rows
Inserted another chunk, took 2.078 seconds
All chunks have been processed. Total rows inserted: {total_rows_processed}


In [39]:
df_1 = pd.read_csv("taxi_zone.csv")

In [41]:
print(pd.io.sql.get_schema(df_1, name="taxi_zone", con=engine))


CREATE TABLE taxi_zone (
	"LocationID" BIGINT, 
	"Borough" TEXT, 
	"Zone" TEXT, 
	service_zone TEXT
)




In [42]:
df_1.head(0)

,LocationID,Borough,Zone,service_zone


In [43]:
df_1.head(0).to_sql(name="taxi_zone", con=engine, if_exists="replace")

0

In [44]:
df_1.to_sql(name='taxi_zone', con=engine, if_exists='append')

265

In [46]:
query_q3 = """
SELECT  
  COUNT(CASE WHEN trip_distance <= 1 THEN 1 END) AS count_a,
  COUNT(CASE WHEN trip_distance > 1 AND trip_distance <= 3 THEN 1 END) AS count_b,
  COUNT(CASE WHEN trip_distance > 3 AND trip_distance <= 7 THEN 1 END) AS count_c,
  COUNT(CASE WHEN trip_distance > 7 AND trip_distance <= 10 THEN 1 END) AS count_d,
  COUNT(CASE WHEN trip_distance > 10 THEN 1 END) AS count_e
FROM 
  green_tripdata
WHERE 
  lpep_pickup_datetime >= '2019-10-01' 
  AND lpep_pickup_datetime < '2019-11-01';
"""

pd.read_sql(query_q3, con=engine)

,count_a,count_b,count_c,count_d,count_e
0,103486,196663,108704,27459,35044


In [47]:
query_q4 = """
SELECT
  lpep_pickup_datetime,
  trip_distance
FROM 
  green_tripdata
ORDER BY
  trip_distance DESC
LIMIT 1;
"""

pd.read_sql(query_q4, con=engine)

,lpep_pickup_datetime,trip_distance
0,2019-10-31 23:23:41,515.89


In [48]:
query_q5 = """
SELECT 
    t."PULocationID", 
    SUM(total_amount) AS total_amount,
	z."Zone" AS pickup_zone
FROM 
    green_tripdata t
JOIN taxi_zone z 
    ON t."PULocationID" = z."LocationID"
WHERE 
    DATE(lpep_pickup_datetime) = '2019-10-18'
GROUP BY 
    t."PULocationID", z."Zone"
HAVING 
    SUM(total_amount) > 13000
ORDER BY 
    total_amount DESC;
"""

pd.read_sql(query_q5, con=engine)

,PULocationID,total_amount,pickup_zone
0,74,18686.68,East Harlem North
1,75,16797.26,East Harlem South
2,166,13029.79,Morningside Heights


In [49]:
query_q6 = """
SELECT
  TO_CHAR(lpep_pickup_datetime, 'YYYY-MM') AS month_year,
  "PULocationID",
  tip_amount AS largest_tip,
  z."Zone" AS "dropoff_loc"
FROM 
    green_tripdata t
JOIN taxi_zone z 
    ON t."DOLocationID" = z."LocationID"
WHERE
	TO_CHAR(lpep_pickup_datetime, 'YYYY-MM') = '2019-10'
	AND "PULocationID" = (
        SELECT "LocationID"
        FROM taxi_zone
        WHERE "Zone" = 'East Harlem North')
ORDER BY 
    largest_tip DESC
LIMIT 1;
"""

pd.read_sql(query_q6, con=engine)

,month_year,PULocationID,largest_tip,dropoff_loc
0,2019-10,74,87.3,JFK Airport
